In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
import tensorflow as tf
from tensorflow import keras

import random
from random import choice
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Concatenate, Embedding, Flatten, Activation, Dropout
from sklearn.model_selection import KFold
from tensorflow.python.client import device_lib
warnings.filterwarnings('ignore')
import random

In [2]:
import gc

## Data Preprocessing

In [3]:
# load the data
import pandas as pd
dataframe = pd.read_csv('../input/practice-log-demographics/Practice_Log_Demographics (1).csv', low_memory=False)
dataframe.head()

,Unnamed: 0,chapter_label,sub_chapter_label,question_name,user_id.x,term,STDNT_SEX_CD,NonNativeEnglish,White,Asian,...,classGraded,classHonors,Pass_Fail,parentsGraduateEdu,minorityGroup,q,day's_available_flashcards,start_practice,end_practice,days_offset
0,1,Sort,intro-SortingwithSortandSorted,ee_sort_04,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,18,2018-03-04 20:00:19,2018-03-04 20:00:35,45
1,2,Functions,Returningavaluefromafunction,test_questionfunctions_2_3,148,WN 2018,1,0,1,0,...,1,0,0,0,0,4,16,2018-02-10 11:57:55,2018-02-10 11:58:35,23
2,3,RESTAPIs,functionswithrestapis,test_question_functions_rest_apis_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,1,2,2018-03-12 17:03:07,2018-03-12 17:04:28,53
3,4,SimplePythonData,FunctionCalls,exercise_functionCalls_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,10,2018-01-30 14:15:49,2018-01-30 14:15:58,12
4,5,DictionaryAccumulation,AccumulatingaMaximumValue,ps_04_07,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,23,2018-03-03 14:04:17,2018-03-03 14:04:25,44


In [4]:
#sort data based on timestamp
dataframe = dataframe.sort_values(by=['start_practice'])

In [5]:
FEATURES = ['chapter_label', 'sub_chapter_label','question_name','user_id','term',
            'STDNT_SEX_CD', 
            'NonNativeEnglish',
            'White',
            'Asian',
            'WhiteOrAsian',
            'Hispanic',
            'AfricanAmerican',
            'OtherEthnicities',
            'NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'gradingType',
            'birthYear',
            'exclClassCumGPA',
            'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'q',
            'available_flashcards', 
            'start_practice', 
            'end_practice',
            'days_offset']

In [6]:
dataframe['available_flashcards'] = dataframe["day's_available_flashcards"][:]
dataframe = dataframe.drop(["day's_available_flashcards"], axis=1)
dataframe['user_id'] = dataframe["user_id.x"][:]
dataframe = dataframe.drop(["user_id.x"], axis=1)

In [7]:
gc.collect()

66

In [8]:
dataframe = dataframe[FEATURES]
gc.collect()

22

In [9]:
#label encode term, chapter_label, question_name, sub_chapter_label columns
dataframe['term'] = dataframe['term'].astype('category')
dataframe['user_id'] = dataframe['user_id'].astype(int)
dataframe['user_id'] = dataframe['user_id'].astype(str)
dataframe['user_id'] = dataframe['term'].str.cat(dataframe['user_id'], sep=':')
dataframe['user_id'] = dataframe['user_id'].astype('category')
dataframe['chapter_label'] = dataframe['chapter_label'].astype('category')
dataframe['sub_chapter_label'] = dataframe['sub_chapter_label'].astype('category')
dataframe['question_name'] = dataframe['question_name'].astype('category')


In [10]:
#calculate time_lag and prev_time_elapsed
dataframe['prev_time_elapsed'] = None
dataframe['time_lag'] = None
dataframe['time_lag'] = dataframe['time_lag'].astype(np.float)
dataframe['prev_time_elapsed'] = dataframe['prev_time_elapsed'].astype(np.float)
dataframe.start_practice = pd.to_datetime(dataframe.start_practice, format='%Y-%m-%d %H:%M:%S')
dataframe.end_practice = pd.to_datetime(dataframe.end_practice, format='%Y-%m-%d %H:%M:%S')

In [11]:
dataframe['dif'] = dataframe.end_practice - dataframe.start_practice
dataframe['dif'] = dataframe['dif'] /np.timedelta64(1, 's')
dataframe['answer_correct'] = np.where((dataframe['q']==5) & (dataframe['dif'] <= 60), 1, 0)

In [13]:
#drop column end_practice
dataframe.drop(columns=['end_practice'], inplace=True)

In [14]:
# calculate the age feature
dataframe['term_value'] = [int(ele[3:]) for ele in dataframe['term']]
dataframe['age'] = dataframe['term_value'] - dataframe['birthYear']

In [15]:
# drop term_value and birthYear column
dataframe.drop(columns=['term_value', 'birthYear'], inplace=True)

In [16]:
# convert minors_count to int value
new_minors_count = []
for i in dataframe['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

dataframe['minorsCount'] = new_minors_count

In [17]:
print("we have ", dataframe['user_id'].nunique()," users in total.")

we have  1110  users in total.


In [18]:
for category in ['term','chapter_label', 'sub_chapter_label', 'question_name']:
  dataframe[category] =  dataframe[category].cat.codes

In [19]:
NUMERIC_FEATURE =  ['age',
            'exclClassCumGPA',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'available_flashcards', 
            'days_offset', 
            'prev_time_elapsed',
             'time_lag']
# z-score normalize the numerical features
for f in NUMERIC_FEATURE:
  m = dataframe[f].mean()
  std = dataframe[f].std()
  dataframe[f] = (dataframe[f] - m)/std

In [20]:
FEATURE_TRANS =  ['answer_correct', 'chapter_label', 'sub_chapter_label','question_name','user_id','term',
                  'STDNT_SEX_CD', 
                    'White','Asian','NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'age',
            'exclClassCumGPA',
           'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'available_flashcards', 
            'days_offset', 'prev_time_elapsed',
             'time_lag']
grouped_data = dataframe[FEATURE_TRANS].groupby(['user_id']).apply(lambda r: (
                r['answer_correct'],
                r['term'],
                r['chapter_label'],
                r['sub_chapter_label'],
                r['question_name'],
                np.array([r['STDNT_SEX_CD'],r['STDNT_CTZN_STAT_CD'], r['international'], 
                  r['White'],r['Asian'],r['NonWhiteOrAsian'],
                 r['age'],r['exclClassCumGPA'],
                r['Freshman'], r['Junior'], r['Sophomore'], r['Senior'],
                r['termCreditsGPA'], r['termCreditsNoGPA'],
                r['athlete_1'], r['honorsPro'],
                r['LSA'], r['programBusiness'], r['programEngineering'], 
                r['programInformation'], r['programOther'],
                r['HSCalculusTaken'],  r['highSchoolGPA'], 
                r['majorsCount'], r['minorsCount'],
                r['PREV_TERM_CUM_GPA'], 
                r['parentsGraduateEdu'], r['minorityGroup'],
                r['available_flashcards'],
                r['days_offset'],
                r['prev_time_elapsed'],
                r['time_lag']
              ]).transpose()
                ))

In [21]:
gc.collect()

22

In [22]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in grouped_data.index:
  if len(grouped_data[index][0]) <= 10:
    toRemove.append(index)
grouped_data = grouped_data.drop(index=toRemove)

In [23]:
#SETTINGS -> can be modified at any time
MAXLENGTH = 300
EMBEDDING_DIM = 128
DENSE_NEURON = 16
LSTM_NEURON = 32

In [24]:
#FEATURES_SIZE = 38
CHAPTER_SIZE = 38
SUB_CHAPTER_SIZE = 223
QUESTION_SIZE = 1069

In [25]:
dataframe['sub_chapter_label'].unique()

array([ 96,  86,  88,  28,  12,  82, 125,  32,  69,  33,  21, 115, 120,
       119,  91,  73,  57,  77, 117,  40,  74,  41,  68,  39,  92,  47,
       104,  95,  53,  49,  50,  16,  52,  55,   7,  15,  51, 102, 103,
        11,  94, 108, 100,  17,  67,  63,  13, 139,  26,  25,   8, 107,
         0,  90,  48,  54, 138,   1,   2,   3,  34,  35,  85,   5,  24,
        37,  36,  78,  70,  45, 142,  71,  10,  89, 111, 112, 109, 116,
        76,  72, 148, 134, 146, 221, 143,  56,  60,  61,  23, 140, 106,
        42,  83,  22, 145, 136, 144,  38,   4,  19, 137,  75,  43,  98,
        84, 113, 173, 174, 175, 176, 177, 178, 180, 181, 182, 187, 189,
       190, 191, 192, 193, 195, 197, 151, 152, 153, 156, 157, 158, 159,
       160, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 162,
       179, 186, 188, 194, 183, 184, 185, 211, 212, 213, 154, 155, 196,
         6, 118,  18, 215,  46, 216, 141,  27, 217, 105, 123, 21

In [26]:
#create dataset class
#to prepare it for train, valid, and test sets
from torch.utils.data import Dataset, DataLoader
class SPACE_DATASET(Dataset):
    def __init__(self, data, maxlength = 300):
        super(SPACE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.users = list()
        for user in data.index:
            self.users.append(user)

    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        user = user
        target, term, ch_label, sub_ch_label, ques_name, features = self.data[user]
        
        #0s should be used as padding values
        ori_target = target.values 
        term = term.values
        ch_label = ch_label.values + 1
        sub_ch_label = sub_ch_label.values +1
        ques_name = ques_name.values + 1
        
        n = len(ch_label)

        # one hot for term
        term_encode = [0]*7
        term_encode[term[0]] = 1
        shifted_target= []

        
        # get  user interaction informations in the previous MAXLEN interactions
        if n > self.maxlength:
          ch_label = ch_label[-self.maxlength:]
          sub_ch_label = sub_ch_label[-self.maxlength:]
          ques_name = ques_name[-self.maxlength:]
          features = features[-self.maxlength:]
          target = ori_target[-self.maxlength:]
          shifted_target = ori_target[ (-self.maxlength - 1) :-1]
        else:
          ch_label = [0]*(self.maxlength - n)+list(ch_label[:])
          sub_ch_label = [0]*(self.maxlength - n)+list(sub_ch_label[:])
          ques_name = [0]*(self.maxlength - n)+list(ques_name[:])
          features = [[0]*len(features[0])]*(self.maxlength  - n)+list(features[:])
          target = [-1]*(self.maxlength - n) + list(ori_target[:])
          shifted_target = [-1]*(self.maxlength + 1 - n) + list(ori_target[:-1])

        new_features = []
        count = 0
        for f in features:
          temp = list(f)
          temp.extend(term_encode)
          # temp.append(shifted_target[count]) #uncomment this line for include previous response feature
          new_features.append(temp)
          count += 1
        features = new_features
        return ch_label,sub_ch_label,ques_name,target

In [27]:
SUB_CHAPTER_SIZE

223

In [28]:
# 5 fold cross validation
#HERE WE ONLY USE
#CHAPTER, SUB_CHAPTER
#QUESTION
X = np.array(grouped_data.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
val_losses = list()
val_aucs = list()
train_eval = list()
test_eval = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train_data_space = SPACE_DATASET(grouped_data[users_train], MAXLENGTH)
    val_data_space = SPACE_DATASET(grouped_data[users_val], MAXLENGTH)
    test_data_space = SPACE_DATASET(grouped_data[users_test], MAXLENGTH)
    #construct training input
    train_chapter=[]
    train_sub_chapter=[]
    train_question = []
    train_features=[]
    train_labels=[]
    for i in range(len(users_train)):
        user = train_data_space.__getitem__(i)
        train_chapter.append(user[0])
        train_sub_chapter.append(user[1]) 
        train_question.append(user[2])
        #train_features.append(user[3])
        train_labels.append(user[3])
    train_chapter = np.array(train_chapter)
    train_sub_chapter = np.array(train_sub_chapter)
    train_question = np.array(train_question)
    #train_features = np.array(train_features)
    train_labels= np.array(train_labels)[..., np.newaxis]

    #construct validation input
    val_chapter=[]
    val_sub_chapter=[]
    val_question = []
    #val_features=[]
    val_labels=[]
    for i in range(len(users_val)):
        user = val_data_space.__getitem__(i)
        val_chapter.append(user[0])
        val_sub_chapter.append(user[1]) 
        val_question.append(user[2])
        #val_features.append(user[3])
        val_labels.append(user[3])
    val_chapter = np.array(val_chapter)
    val_sub_chapter = np.array(val_sub_chapter)
    #val_features = np.array(val_features)
    val_question = np.array(val_question)
    val_labels= np.array(val_labels)[..., np.newaxis]

    # construct test input
    test_chapter=[]
    test_sub_chapter=[]
    #test_features=[]
    test_question=[]
    test_labels=[]
    for i in range(len(users_test)):
        user = test_data_space.__getitem__(i)
        test_chapter.append(user[0])
        test_sub_chapter.append(user[1]) 
        test_question.append(user[2])
        #test_features.append(user[3])
        test_labels.append(user[3])
    test_chapter = np.array(test_chapter)
    test_sub_chapter = np.array(test_sub_chapter)
    #test_features = np.array(test_features)
    test_question = np.array(test_question)
    test_labels= np.array(test_labels)[..., np.newaxis]

    # define loss function and evaluation metrics
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    acc = tf.keras.metrics.Accuracy()
    auc = tf.keras.metrics.AUC()

    def masked_bce(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return bce(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_acc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      flat_pred = (flat_pred >= 0.5)
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return acc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_auc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return auc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    # input layer
    input_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_sub_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_ques =  tf.keras.Input(shape=(MAXLENGTH))
    #input_features = tf.keras.Input(shape=(MAXLENGTH, FEATURES_SIZE))

    # embedding layer for categorical features
    embedding_chap = Embedding(input_dim = CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_chap)
    embedding_sub_chap = Embedding(input_dim = SUB_CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_sub_chap) 
    embedding_ques = Embedding(input_dim = QUESTION_SIZE, output_dim = EMBEDDING_DIM)(input_ques)       

    # dense layer for numeric features
    #dense_features = Dense(EMBEDDING_DIM,input_shape = (None, MAXLENGTH))(input_features)

    # definr lstm layers
    lstm_chap = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_chap)
    lstm_sub_chap = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_sub_chap)
    lstm_ques = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(embedding_ques)
    #lstm_features = LSTM(LSTM_NEURON, input_shape = (None, EMBEDDING_DIM),return_sequences = True)(dense_features)

    lstm_output = tf.concat([lstm_chap, lstm_sub_chap, lstm_ques], axis = 2)

    dense1 = Dense(256, input_shape = (None, 3*EMBEDDING_DIM), activation='relu')(lstm_output)
    dropout1 = Dropout(0.1)(dense1)
    dense2 = Dense(64, input_shape = (None, 256), activation='relu')(dropout1)
    dropout2 = Dropout(0.1)(dense2)
    pred = Dense(1, input_shape = (None, 64), activation='sigmoid')(dropout2)

    model = tf.keras.Model(
        inputs=[input_chap, input_sub_chap,input_ques],
        outputs=pred,
        name='lstm_model'
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt_adam = Adam(learning_rate = 0.005)
    model.compile(
        optimizer=opt_adam,
        loss= masked_bce,
        metrics = [masked_acc, masked_auc]
    )

    history = model.fit(
      [train_chapter, train_sub_chapter, train_question],
      train_labels,
      batch_size = 64,
      epochs = 100,
      validation_data=([val_chapter, val_sub_chapter, val_question], val_labels),
      callbacks=[callback]
    )
    val_losses.append(list(history.history['val_loss']))
    train_losses.append(list(history.history['loss']))
    val_aucs.append(list(history.history['val_masked_auc']))
    train_aucs.append(list(history.history['masked_auc']))
    train_score = model.evaluate([train_chapter, train_sub_chapter, train_question], train_labels)
    train_eval.append(train_score)
    test_score = model.evaluate([test_chapter, test_sub_chapter, test_question], test_labels)
    test_eval.append(test_score)
    print("Test: ", test_score)
    def reset_weights(model):
      for layer in model.layers: 
        if isinstance(layer, tf.keras.Model):
          reset_weights(layer)
          continue
        for k, initializer in layer.__dict__.items():
          if "initializer" not in k:
            continue
          # find the corresponding variable
          var = getattr(layer, k.replace("_initializer", ""))
          var.assign(initializer(var.shape, var.dtype))
    reset_weights(model)

Epoch 1/100
14/14 [==============================] - 15s 638ms/step - loss: 0.6039 - masked_acc: 0.5719 - masked_auc: 0.5188 - val_loss: 0.5086 - val_masked_acc: 0.7044 - val_masked_auc: 0.6302
Epoch 2/100
14/14 [==============================] - 7s 479ms/step - loss: 0.4914 - masked_acc: 0.7089 - masked_auc: 0.6610 - val_loss: 0.4714 - val_masked_acc: 0.7209 - val_masked_auc: 0.7082
Epoch 3/100
14/14 [==============================] - 7s 468ms/step - loss: 0.4651 - masked_acc: 0.7259 - masked_auc: 0.7218 - val_loss: 0.4578 - val_masked_acc: 0.7369 - val_masked_auc: 0.7437
Epoch 4/100
14/14 [==============================] - 7s 474ms/step - loss: 0.4459 - masked_acc: 0.7400 - masked_auc: 0.7499 - val_loss: 0.4540 - val_masked_acc: 0.7463 - val_masked_auc: 0.7621
Epoch 5/100
14/14 [==============================] - 6s 449ms/step - loss: 0.4384 - masked_acc: 0.7482 - masked_auc: 0.7665 - val_loss: 0.4502 - val_masked_acc: 0.7523 - val_masked_auc: 0.7742
Epoch 6/100
14/14 [===============

In [29]:
t_eval = np.array(test_eval)
print("test avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("test avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("test avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

test avg loss:  0.4418660640716553 +/- 0.013666928472421496
test avg acc:  0.7798750281333924 +/- 0.003059035121383781
test avg auc:  0.8279644250869751 +/- 0.005179175003826304


In [30]:
t_eval = np.array(train_eval)
print("train avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("train avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("train avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

train avg loss:  0.38875927925109866 +/- 0.005123665065537777
train avg acc:  0.779647171497345 +/- 0.003105376418295783
train avg auc:  0.8273740649223328 +/- 0.0053446471728854665
